---

# <u> **The Naïve, Barnes-Hut, and Fast Multipole Methods**  </u>
### *Implementation, Analysis, and Evaluation of different Algorithms for $N$-Body Simulations*  

**Author:** Hugo Robijns

**Date:** April 2025 

**Abstract:**

---

How this report is formatted:  

In [ ]:
# necessary libraries for the running of this report
import pandas as pd
from IPython.display import Image, Video, display
from IPython.core.display import HTML

___

### **Introduction:**
The analysis of many-body systems is highly relevant to many fields, from celestial bodies interracting gravitationally, to vortex methods in computational fluid dynamics, and more [CITE]. These systems are complex to simulate: for three or more bodies the system becomes chaotic and extremely difficult to approach analytically without major simplifications, and naïvely calculating pairwise interractions for $N$ bodies leads to a numerical complexity of $\mathcal{O}(N^2)$, which scales poorly.  

&emsp; &emsp; This report aims to analyse and implement two algorithms which attempt to reduce this complexity, the Barnes-Hut and fast multipole method (FMM) techniques, alongside the brute-force $\mathcal{O}(N^2)$ approach, in order to compare their performances. For the sake of comparison, we will base analysis around the simple 2D example of particles under the influence of a logarithmic potential.

&emsp; &emsp; The report begins with a section analysing the algorithms and their complexity, followed by a section describing their implementation, and finally a discussion section. We find that: 

&emsp; &emsp; All accompanying code is found in this repository.


_Figure 1: a simulation of X_

---
## **1. Analysis of Algorithms**  


### *1.1 Naïve Pairwise Calculation, $\mathcal{O}(N^2)$*

In a 2D system, a particle $i$ of charge $q_i$ situated at $\vec{x}_i = (x_i,y_i)$ produces a electrostatic potential $\phi_{i}(x,y)$ and field $E_{i}(x,y)$ given by (up to a scale factor):
$$\phi_{i}(x,y)=-q_i\log(|\vec{x}-\vec{x}_i|)$$
$$E_{i}(x,y) = -\nabla_{2\text{D}}\phi_{x_0}(x,y) = q_i\frac{(\vec{x}-\vec{x}_i)}{|\vec{x}-\vec{x}_i|^2}$$
&emsp; &emsp; therefore for an $N$-body system, we must sum over the fields produced by all the other particles, and the equations of motion become:
$$m_i \frac{\text{d}^2\vec{x}_i}{\text{d}t^2}  = q_i \sum_{j=1, j\neq i}^{N} \frac{q_j (\vec{x}_j-\vec{x}_i)}{|(\vec{x}_j-\vec{x}_i)|^2}$$ 

&emsp; &emsp; from which the position and velocity of the particles can be updated through integration.

It is clear to see that in order to simulate such a system directly, we must carry out $N-1$ calculations for each of the $N$ particles (or half of the particles, if exploiting Newton's $3^\text{rd}$ law), leading to a theoretical complexity of $\mathcal{O}(N^2)$. Note also that the force between two particles diverges as they get close to each other: this requires the inclusion of a softening parameter $\epsilon$ which will be discussed further in Section 2.1. 

### *1.2 Barnes-Hut Algorithm, $\mathcal{O}(N\log N)$*  
The Barnes-Hut Algorithm, introduced by Josh Barnes and Piet Hut in 1986 [CITE] attempts to reduce complexity by treating distant clusters of particles as a single body. It does this through hierarchical decomposition, in this case the use of tree data structures, specifically an quadtree in 2D. 

##### 1.2.1 Tree Construction
Beginning with a topmost node that represents the whole simulation space, a quadtree is formed by subdividing the space into 4 quadrants recursively until each body lies in its own leaf node, a node with no children (see Figure 2). Each node, both internal and leaf nodes, contains information about the charge distribution (specifically the total charge and centre of charge) of the node. Formally, they are constructed by inserting particles into the data structure one after the other:

1. if the node into which the particle is being placed is an empty leaf node, the particle is simply placed there and the total charge and centre of charge position of the node are trivially updated.

2. if the node is an internal node (i.e. a node with children), the total charge and centre of charge position of the node are updated, and the particle moves down a level in the tree to the appropriate quadrant.

3. if the node is a leaf node which already contains a particle, the total charge and centre of charge position of the node is updated, and the node is subdivided into 4 quadrants. Then, both particles are placed into their appropriate quadrants as above, which may require further subdivision if they end up in the same quadrant again.   


In [4]:
HTML('<div style="text-align: center;"><img src="figures/quadtree_plot.png" width="800"/></div>')

_Figure 2: a visualisation of the quadtree constructed for 10 particles in a 2D simulation. The square grid-lines seen in the left plot are the spatial borders of the leaf nodes, the smallest subdivisions of the tree which have no children of themselves, and therefore contain either 0 or 1 particles. The figure on the right shows a schematic of the quadtree: hollow markers indicate empty nodes, and solid markers indicate nodes containing particles._

##### 1.2.2 Force Calculation
To calculate forces, the tree is recursively traversed from the root for each particle:

1. If the node is a leaf node and contains a particle that is not itself, then the force is calculated in the usual way.

2. If the node is an internal node, the ratio: $$\frac{\text{size of node in space (i.e. side length of the cube)}}{\text{distance from particle to centre of charge of node}} \equiv \frac{s}{d} $$ is calculated. If this is less than a threshold value $\theta$ (usually taken to be $\sim$ 1), then the node is seen to contain particles that are closely packed and far away, and as a result can be treated as one single particle, ignoring its children nodes. The force on the particle is calculated in the usual way, using the centre of charge position and total charge of the node.

3. If $s/d > \theta$, the process recursively continues for each of the children of the node, until reaching the leaf node or a node which satisfies the condition.

These forces are then summed to give an overall acceleration for the particle. It is clear to see that $\theta$ determines the accuracy of the simulation: a larger $\theta$ leads to fewer calculations but a more approximate solution, whilst a smaller $\theta$ may take longer but provide a more reliable solution. In the limit $\theta\rightarrow0$ the algorithm reduces to the brute-force approach, since no internal nodes are treated as single particles.

##### 1.2.3 Complexity
For a _balanced_ quadtree with $h$ levels, the bottom level has $4^h$ nodes. Therefore:
$$ 4^h \geq N \Rightarrow h \sim \log_4 N = \frac{\log_2 N}{\log_2 4} = \frac{1}{2} \log_2 N \text{ or } \mathcal{O}(\log N)$$
Insertion of each of the $N$ particles requires travelling the $h$ levels of the tree from root to leaf, therefore the overall complexity of constructing a quadtree is given by $\mathcal{O}(N\log N)$. 

The complexity of the force calculation step is more complicated. A typical reasoning (as laid out in the original paper) follows as such: again assuming a balanced quadtree (i.e. homogoneously distributed charge), consider quadrupling the number of particles. This is equivalent to adjoining 3 root nodes to the existing one. To calculate the force on a particle situated in the 'original tree', these additional particles will contribue a fixed number of extra calculations, which is a function of $\theta$ but not $N$. To imagine this, consider $\theta \gg 1$: this will simply add 3 force calculations, since the centre of charge of all three of the the 'additional root nodes' are far enough away, and they can be treated as a single particle. Reducing $\theta$ to more reasonable values will increase the number of children of these 'additional nodes' that need to be explored, but assuming $N$ is large enough (and this further exploration is therefore not reaching leaf nodes), then the number of additional calculations does not depend on $N$. Since the number of force calculations is increasing by an additive constant (a function of $\theta$) whilst $N$ is increasing by a multiplicative factor, the complexity goes as $\mathcal{O}(\log N)$ for a single particle, or $\mathcal{O}(N\log N)$ for $N$ particles. 

Summing the tree construction and force calculation complexities leads to overall complexity $\mathcal{O}(N\log N)$, a major improvement on the naive $\mathcal{O}(N^2)$.


### *1.3 Fast Multipole Method (FMM), $\mathcal{O}(N)$* 
The FMM algorithm was introduced by Leslie Greengard and Vladimir Rokhlin Jr. in the 1980s [CITE]. It uses similar hierarchical decomposition methods as the Barnes-Hut algorithm, but instead works with more accurate local and multipole expansions of the potential rather than simple information on the centre of charge of a node. Re-use of computation by traversing the tree in both directions allows complexity to be reduced to $\mathcal{O}(N)$ for arbitrary precision. For the sake of this report, detailed derivation of the maths will not be carried out: see the original paper for more detailed discussion. 

A high-level overview of the algorithm is as follows:

1. A quadtree is constructed as in the Barnes-Hut method, however the approach need not be as rigorous, since the algorithm simply requires leaf nodes to house a small, constant ($\mathcal{O}(1)$) number of bodies.

2. The tree is traversed from bottom to top, calculating the multipole expansion for the potential produced by each node at a point far away.

3. The tree is traversed from top to bottom, calculating the local expansion, or the potential felt at each node due to other nodes far away.

4. At the bottom 'leaf' level, direct pair-wise interractions are calculated with nearest neigbours - since in step 1. we ensured each leaf node had a small number of bodies, this is inexpensive.

##### 1.3.1 Multipole Expansions and Translations
Expressing the logarithmic potential from before produced by a particle $i$ in terms of complex variables $(x,y) \in \mathcal{R}^2 \rightarrow z = x+iy \in \mathcal{C}$, we get:
$$\phi_{i}(x,y) \rightarrow \phi_{i}(z) = \text{Re}(-\log(z-z_0))$$
&emsp; &emsp; which for $|z|>|z_0|$ can be expanded as:
$$ \phi_{i}(z) = q \left(\log(z) - \sum_{k=1}^\infty \frac{1}{k} \left( \frac{z_0}{z} \right)^k \right) $$
Putting together the potential, as expressed above, from $N$ particles of charge $\{q_i, i = 1 \ldots N\}$ at positions $\{z_i, i = 1 \ldots N\}$, gives us the _multipole expansion_:
$$\phi(z) = Q\log(z) + \sum_{k=1}^\infty \frac{a_k}{z^k} \phantom{xxx}\text{ where } Q = \sum_{i=1}^{N} q_i \text{ and } a_k = \sum_{i=1}^{N}\frac{-q_iz_i^k}{k}$$
The efficiency in the FMM algorithm comes in part due to its re-use of computation of the above expansion, through the use of translations. These allow expansion coefficients for adjacent level nodes to be calculated more cheaply. The three relevant translations are the multipole-to-multipole shift (M2M translation):
$$ \phi(z) = Q \log (z) + \sum_{l=1}^\infty \frac{b^l}{z}  \phantom{xxx}\text{ where } b_l = \sum_{k=1}^l a_k z_0^{l-k} \binom{l-1}{k-1} -\frac{a_0 z_0^l}{l} $$
&emsp; &emsp; the multipole-to-local shift (M2L translation):
$$ \phi(z) = \sum_{l=0}^{\infty}b_l z^l  \phantom{xxx}\text{ where } b_0 = \sum_{k=1}^\infty (-1)^k \frac{a_k}{z_0^k} + a_0 \log(-z_0) \text{ and } b_l = \frac{1}{z_0^l} \sum_{k=1}^\infty \frac{(-1)^k a_k}{z_0^k}\binom{l+k-1}{k-1} - \frac{a_0}{lz_0^l}$$
&emsp; &emsp; and the local-to-local shift (L2L translation):
$$ \sum_{k=0}^N a_k(z-z_0)^k = \sum_{l=0}^N \left(\sum_{k=l}^N a_k \binom{k}{l} (-z_0)^{k-l} \right)z^l$$


##### 1.3.2 Algorithm and Complexity
Starting from the leaf nodes, the multipole expansidon for each node is calculated using the above analytical expression. Since we are essentially aggregating (summing) over the charges, this step is $\mathcal{O}(N)$. Going up the tree, the expansion of each node can be formed through aggregating the expansion of its children, using the M2M translation and summing. 

Working downards from the top of the tree, we calculate the local (Taylor) expansion of the potential experienced felt at each node. Firstly, the local expansion of the parent node (if it exists) is shifted to the centre of the new node using a L2L translation. Then, the multipole expansion of non-adjacent (far away) nodes is translated into a local expansion at the location of the node, in a M2L translations.

<center> **IMAGE** </center>



Finally, upon reaching the bottom leaf level, we calculate the direct contributions from the few remaining bodies, which is not computationally expensive since there are a small number of bodies at this level. 

---

## **2. Implementation and Performance**

All three algorithms were implemented in vanilla Python alongside some basic open-source libraries, mainly _Numpy_ and _Matplotlib_, and modules _time_ and _collections_. For timing comparison, the results in this report were produced by running the algorithms on a Macbook Air with CPU 1.1 GHz Quad-Core Intel Core i5. Note that, since the aim of the report was to compare performance and complexities, all simulations were carried out with arbitrary simulation units (Coulomb constant $k=1$, $q=1$ etc.). 

### *2.1 Naïve Pairwise Calculation*
The brute force implementation is found in source/naive. Discussion surrounding the implementation of the softening parameter, time step and integration techniques are relevant for the other algorithms too. The main pairwise force calculation is shown below.


In [6]:
def compute_forces(self):
        """Compute pairwise repulsive Coulomb forces between all bodies."""
        # reset forces
        for b in self.bodies:
            b.force = np.zeros(2)

        # pairwise loop (i<j to avoid double counting)
        for i, b1 in enumerate(self.bodies):
            for j in range(i+1, len(self.bodies)):
                b2 = self.bodies[j]
                # vector from b2 to b1 (repulsion)
                diff = b1.position - b2.position
                dist = np.linalg.norm(diff)
                # Coulomb force magnitude
                f_mag = k * b1.charge * b2.charge / (dist**2 + soft**2)
                # unit direction vector
                f_vec = (f_mag / (dist + 1e-16)) * diff
                # apply equal and opposite forces
                b1.force +=  f_vec
                b2.force += -f_vec


##### 2.1.1 Softening Parameter ($\epsilon$) and Time Step ($\Delta t$)
In order to avoid divergence and instability when particles become close, we introduce a softening parameter $\epsilon$ such that:
$$\phi_{i}(x,y)=-q_i\log(|\vec{x}-\vec{x}_i|)\phantom{x}\xrightarrow{\text{softening}} \phantom{x}\phi_{i}(x,y)=-q_i\log(\sqrt{|\vec{x}-\vec{x}_i|^2 + \epsilon^2})$$ 
$$E_{i}(x,y) = -\nabla_{2\text{D}}\phi_{x_0}(x,y) = q_i\frac{(\vec{x}-\vec{x}_i)}{|\vec{x}-\vec{x}_i|^2}\phantom{x}\xrightarrow{\text{softening}}\phantom{x} E_{i}(x,y)  = q_i\frac{(\vec{x}-\vec{x}_i)}{|\vec{x}-\vec{x}_i|^2+\epsilon^2}$$
$\epsilon$ should be chosen to be small enough to keep the simulation accurate, but large enough to avoid singularities. This depends on the scale of the system, and the time-step, $\Delta t$. A demonstration of the influence of $\epsilon$ is shown below.

In [ ]:
display(Video("figures/softening.mp4", embed=True, width=640, height=360))

_Figure X: a demonstration of the effect of the softening parameter $\epsilon$ showing an animation of three bodies interracting (left), and the percentage total energy change of the system from its initial value for three different values of $\epsilon$ (right). For smaller values of $\epsilon$, the close passes cause larger variations away from the initial, conserved energy of the system (which will be different for the different values of $\epsilon$). However, for larger values of $\epsilon$, the system is less true to reality. Note that for this simulation, in contrast to the others in this report, an attractive potential has been used to encourage close passes._

Work has been carried out in order to quantify the most reliable size of $\epsilon$ and $\Delta t$ for a system, often based on how far particles travel in a time-step in relation to their proximity [CITE]. Indeed, a good way forward is to use an adapative $\Delta t$, which can become smaller as bodies become closer to each other, in order to accurately and stably explore regions of high accelaration. Since this project is less about dealing with collisions, and more about analysing the complexities and performances of the algorithms in scaling to large numbers of bodies, a simple comparison and tuning process was carried out, settling on constant values $\epsilon \sim 0.5$, and $\Delta t \sim 0.01$. These values were kept consistent and used for the other two algorithms as well. In addition, for our complexity analysis we consider systems wherein the particles are well-separated, to avoid any unnecessary instability.


##### 2.1.2 Integration Techniques
Upon calculating the individual acceleration each particle experiences, we integrate to find their updated velocity and position. There are many different possible integration techniques, but we used the symplectic 'kick-drift-kick' technique due to its easy implementation and improved energy conservation compared to the simple Euler step. It is a form of Verlet-velocity integration, and can be implemented through the following equations:


1. $x(t+\Delta t) =x(t)+v(t)\Delta t+\frac{1}{2}a(t)(\Delta t)^2$

2. calculate $a(t+\Delta t)$

3. $v(t+\Delta t) = v(t) + \frac{1}{2}\left(v(t)+v(t+\Delta t)\right)\Delta t$

&emsp; &emsp; where $x$ is position, $v$ is velocity, $a$ is acceleration, and $t$ is time. Further discussion on the derivation, error and stability of this algorithm can be found in [CITE]. Our implementation is shown below, as well as a figure showing the energy drift over time for a simple Euler step compared to 'kick-drift-kick'. 


In [7]:
def move(self):
        # half-kick
        for b in self.bodies:
            b.velocity += 0.5 * (b.force / b.mass) * dt
        # drift
        for b in self.bodies:
            b.position += b.velocity * dt
        # recompute forces
        self.compute_forces()
        # half-kick
        for b in self.bodies:
            b.velocity += 0.5 * (b.force / b.mass) * dt


##### 2.1.2 Computation Time

In [20]:
import pandas as pd
df = (
    pd.read_csv('data/naive.csv')
      .query("N in [10, 100, 1000]")
      .loc[:, [
          'N',
          'naive_time_mean', 'vector_time_mean',
          'naive_mem_mean',  'vector_mem_mean'
      ]]
      .rename(columns={
          'naive_time_mean':  'Time, A (s)',
          'vector_time_mean': 'Time, B (s)',
          'naive_mem_mean':   'Memory, A (MB)',
          'vector_mem_mean':  'Memory, B (MB)'
      })
      .round({
          'Time, A (s)':  6,
          'Time, B (s)':      6,
          'Memory, A (MB)': 3,
          'Memory, B (MB)':     3
      })
)

print(df.to_markdown(index=False, tablefmt='fancy_grid'))

╒══════╤═══════════════╤═══════════════╤══════════════════╤══════════════════╕
│    N │   Time, A (s) │   Time, B (s) │   Memory, A (MB) │   Memory, B (MB) │
╞══════╪═══════════════╪═══════════════╪══════════════════╪══════════════════╡
│   10 │      0.013397 │      0.001248 │            0.002 │            0.012 │
├──────┼───────────────┼───────────────┼──────────────────┼──────────────────┤
│  100 │      1.01081  │      0.009527 │            0.011 │            0.753 │
├──────┼───────────────┼───────────────┼──────────────────┼──────────────────┤
│ 1000 │    125.864    │      0.779014 │            0.108 │           68.744 │
╘══════╧═══════════════╧═══════════════╧══════════════════╧══════════════════╛


As expected, the complexity scales as $N$. Within this approach there are many ways to speed up computation. A simple, one is vectorisation...

### *2.2 Barnes-Hut*
The Barnes-Hut implementation is found in source/BH. The main differences between the Barnes-Hut and the naïve approach is the data structure (quadtree) used to inform which force calculations should be carried out: beyond that (the force calculation itself, integration etc.) the impementation practically identical.

##### 2.2.

##### 2.2.1 Complexity

### *2.1 FMM (see: source/FMM)*


## **3. Results and Discussion**


### _3.1 Complexities as a function of N_
- showing number of calculations as a function of N
- showing time and memory as a function of N


### _3.2 Accuracy (conservation of energy, linear momentum and angular momentum)_

### _3.3 Extensions_
##### 3.2.1 Different Problems and 3D
##### 3.2.2 Vectorisation and parallelisation
---

## Bibliography  

1. J. Barnes and P. Hut, *A Hierarchical O(N log N) Force-Calculation Algorithm*, Nature, 1986.  
2. L. Greengard and V. Rokhlin, *A Fast Algorithm for Particle Simulations*, Journal of Computational Physics, 1987.  
3. D. J. Griffiths, *Introduction to Electrodynamics*, Cambridge University Press, 2017.  